# Introduction

Analysis of a set of simulation outputs that are the result of a parameter sweep.

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os.path
import cPickle as pickle

%load_ext bgcellmodels.extensions.jupyter.skip_cell_extension

In [ ]:
# Width of the page for calibrating fig_size.
# Approx. 16 for matplotlib backend %inline  8 for %notebook
if matplotlib.get_backend() == 'nbAgg':
    from bgcellmodels.extensions.jupyter import jupyterutil
    jupyterutil.notebook_show_figs_after_exception() # fix bug for notebook backend where figures not shown
    page_width = 10
else:
    page_width = 14
ax_height = 3

# Style of figures (default colors etc.): see https://matplotlib.org/gallery/style_sheets/style_sheets_reference.html
plt.style.use('default')

# Load Data

In [ ]:
# Paste output directories below (e.g use ctrl+c in Nautilus file manager)
outputs_clipboard = """
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780629.sonic-head_DA-depleted-v3_CTX-favg14_fburst5
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780630.sonic-head_DA-depleted-v3_CTX-favg14_fburst7
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780631.sonic-head_DA-depleted-v3_CTX-favg14_fburst9
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780632.sonic-head_DA-depleted-v3_CTX-favg14_fburst11
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780633.sonic-head_DA-depleted-v3_CTX-favg14_fburst13
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780634.sonic-head_DA-depleted-v3_CTX-favg14_fburst15
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780635.sonic-head_DA-depleted-v3_CTX-favg14_fburst17
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780636.sonic-head_DA-depleted-v3_CTX-favg14_fburst19
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780637.sonic-head_DA-depleted-v3_CTX-favg14_fburst21
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780638.sonic-head_DA-depleted-v3_CTX-favg14_fburst23
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780639.sonic-head_DA-depleted-v3_CTX-favg14_fburst25
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780640.sonic-head_DA-depleted-v3_CTX-favg14_fburst27
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780641.sonic-head_DA-depleted-v3_CTX-favg14_fburst29
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780642.sonic-head_DA-depleted-v3_CTX-favg14_fburst31
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/q1_const-rate_vary-freq/2018.08.02_job-780643.sonic-head_DA-depleted-v3_CTX-favg14_fburst50
"""
output_dirs = outputs_clipboard.strip().split('\n')

In [ ]:
# Analysis results for each simulation were saved as a dict in pickle format.
# Load them and make them accessible using the value of the sweep variable as key.
analysis_results= {}
sweep_var_name = None
custom_outputs = { # Entries in output_dirs that do not have the correct sweep variable name/value
    # 1 : 2.5, # SETPARAM: custom value for sweep var
}
for i, odir in enumerate(output_dirs):
    exported_results = os.path.join(odir, 'analysis_results.pkl')
    with open(exported_results, 'rb') as f:
        results_dict = pickle.load(f)
        if i in custom_outputs.keys():
            sweep_value = custom_outputs[i]
        elif sweep_var_name is None:
            sweep_var_name = results_dict['sweep_var_name']
            sweep_value = results_dict['sweep_var_value']
#         elif sweep_var_name == results_dict['sweep_var_name']:
#             sweep_value = results_dict['sweep_var_value']
#         else:
#             raise ValueError(
#                 "Simulation results in {} do not have the intended sweep variable!"
#                 "\nMaybe one of the output directories you copied does not match the series.".format(odir))
        # Extract sweep value from filename
        import re
        match = re.search(r'[0-9]+$', odir)
        sweep_value = float(match.group(0))
        # Use saved sweep value
        analysis_results[sweep_value] = results_dict

In [ ]:
# SETPARAM: description of sweep variable for figure legends
sweep_var_legend = "$f_{burst,CTX}$" # LaTeX expressions allowed, double backslashes (\ -> \\)

# SETPARAM: whether cortical bursting input is given
without_ctx_bursting = True

print(analysis_results.keys())
print(analysis_results.values()[0].keys())

# Firing Rates

In [ ]:
# Plot mean firing rates
sweep_vals = np.array(sorted(analysis_results.keys()))
rates = [analysis_results[val]['mean_rate']['CTX'] for val in sweep_vals]
print("Firing rate for each sweep value:\n\n" + 
      "\n".join(["{} : {}".format(a,b) for a,b in zip(sweep_vals, rates)]))

fig, ax = plt.subplots(figsize=(0.5*page_width, ax_height))

ax.plot(sweep_vals, rates, color='g', alpha=0.4)
# ax.plot(sweep_vals, rates, color='r', marker='+')
ax.bar(sweep_vals, rates, width=0.1, alpha=0.4, color='g')

ax.set_xlabel(sweep_var_legend)
ax.set_ylabel('Mean firing rate (Hz)')
ax.set_title('Mean CTX firing rate for parameter sweep "{}"'.format(sweep_var_legend))

ax.set_xticks(sweep_vals)
# ax.set_xticklabels(sweep_vals)

ax.set_yticks(np.arange(0, int(max(rates)+2), 5), minor=False)
ax.set_yticks(np.arange(0, int(max(rates)+2), 1.0), minor=True)
ax.grid(True, axis='y', which='major')

# PSD

## PSD Shared Axis

In [ ]:
# Plot the PSD
sig_label = 'STN_Vm'
sweep_vals = np.array(sorted(analysis_results.keys()))

# Sampling colors from linear scale:
# https://stackoverflow.com/questions/8931268/using-colormaps-to-set-color-of-line-in-matplotlib
cmap = plt.get_cmap('cool')

# Line colors equidistantly on color scale, independent of sweep values
line_colors = [cmap(x) for x in np.linspace(0, 1, len(analysis_results))]
def get_line_color(sweep_val):
    return cmap(float(sweep_val) / sweep_vals.max)

# Linear map of sweep values to color scale
cNorm  = matplotlib.colors.Normalize(vmin=sweep_vals[0], vmax=sweep_vals[-1])
scalarMap = matplotlib.cm.ScalarMappable(norm=cNorm, cmap=cmap)
def get_line_color(sweep_val):
    return scalarMap.to_rgba(sweep_val)

def compare_psd(sig_label, legend=True, colorbar=False, width=0.75*page_width, height=ax_height):
    """ Plot PSD for given signal for each simulation / sweep variable. """
    if colorbar:
        ncols = 2
        # axes for colorbar must be smaller
        subplot_kwargs = {'gridspec_kw': {'width_ratios':[20, 1]}}
    else:
        ncols = 1
        subplot_kwargs = {}
    
    fig, axes = plt.subplots(1, ncols, figsize=(width, height), **subplot_kwargs)
    if colorbar:
        ax = axes[0]
    else:
        ax = axes
    
    # Plot PSD as line for  each sweep value
    sweep_vals = np.array(sorted(analysis_results.keys()))
    lines = []
    for i, sweep_value in enumerate(sweep_vals):
        results = analysis_results[sweep_value]
        freqs, psd = results['PSD'][sig_label]
        plotted = ax.plot(freqs, psd,
                          color=get_line_color(sweep_value), 
                          label='{} = {}'.format(sweep_var_legend, sweep_value))
        lines.extend(plotted)
    
    ax.set_ylabel('Power ({})'.format(psd.units))
    ax.set_xlabel('frequency ({})'.format(freqs.units))
    ax.set_xticks(np.arange(0,55,5), minor=False)
    ax.set_xticks(np.arange(0,51,1), minor=True)
    ax.grid(True)
    ax.set_xlim((0, 35))
    # ax.set_yscale('log')
    ax.set_title('Welch PSD for {}'.format(sig_label))
    if legend:
        if len(sweep_vals) < 7:
            ax.legend(loc='upper right', bbox_to_anchor=(1,1))
        else:
            lower = slice(7)
            upper = slice(7, len(sweep_vals))
            labels = ['{} = {}'.format(sweep_var_legend, v) for v in sweep_vals]
            # 'loc' is corner of legend anchored to anchorpoint
            # 'bbox_to_anchor' is location of anchorpoint on axis bounding box
            leg1 = ax.legend(lines[lower], labels[lower], loc=1, bbox_to_anchor=(.82,1))
            leg2 = ax.legend(lines[upper], labels[upper], loc=2, bbox_to_anchor=(.82,1))
            ax.add_artist(leg1) # otherwise last legend is used
    if colorbar:
        ax = axes[1]
        cb = matplotlib.colorbar.ColorbarBase(ax, cmap=cmap, norm=cNorm, orientation='vertical')
        cb.set_label(sweep_var_legend)
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel

In [ ]:
compare_psd('STN_Vm', legend=True, colorbar=False, width=14, height=4.5)
compare_psd('GPE_Vm')

## PSD Heatmap

In [ ]:
def compare_psd_heatmap(sig_label, x_axis_equidistant=True):
    """
    Compare PSD across parameter sweep using heatmap visualization.
    
    @param  x_axis_equidistant : bool
            If true, x axis will be spaces equidistanctly, and missing values
            will be filled by black bands.
    """
    
    # Concatenate PSDs of different sweep values into matrix
    test_freqs, test_psd = analysis_results.values()[0]['PSD'][sig_label]
    freq_res = test_freqs[1] - test_freqs[0]
    fmax = 100.0
    nfreq = int(fmax/freq_res) + 1

    sweep_vals = np.array(sorted(analysis_results.keys()))
    d_sweep = min(np.diff(sweep_vals)) # resolution of sweep variable
    sweep_axis_spaced = np.arange(sweep_vals.min(), sweep_vals.max()+d_sweep, d_sweep)
    sweep_axis_full = list(sweep_axis_spaced)
    for v in sweep_vals: # steps may miss actual values
        if not np.any(np.isclose(v, sweep_axis_full)):
            sweep_axis_full.append(v)
    if x_axis_equidistant:
        sweep_axis = np.array(sorted(sweep_axis_full))
    else:
        sweep_axis = sweep_vals
    freq_axis = test_freqs[:nfreq]
    sweep_psds = np.empty((nfreq, sweep_axis.size), dtype=float)

    # Fill matrix
    valid_columns = []
    for sweep_value, results in analysis_results.iteritems():
        # col_id = int((sweep_value - sweep_axis.min())/d_sweep)
        col_id = np.where(np.isclose(sweep_value, sweep_axis))[0][0]
        valid_columns.append(col_id)
        freqs, psd = results['PSD'][sig_label]
        sweep_psds[:, col_id] = psd.magnitude[:nfreq]

    # Fill invalid columns with NaN
    invalid_columns = [i for i in range(sweep_axis.size) if i not in valid_columns]
    sweep_psds[:, invalid_columns] = np.nan
    psd_sweep = np.ma.masked_array(sweep_psds, np.isnan(sweep_psds)) # mark invalid/missing data

    # Draw heat map
    cmap = plt.get_cmap('viridis') # e.g. jet, plasma: https://matplotlib.org/examples/color/colormaps_reference.html
    cmap.set_bad(color='k') # color for masked values

    fig, ax = plt.subplots(figsize=(0.5*page_width, ax_height))
    sweep_plotaxis = np.concatenate((sweep_axis, [sweep_axis.max()+d_sweep])) # fix bug/feature in pcolormesh
    plt.pcolormesh(sweep_plotaxis, freq_axis, psd_sweep, cmap=cmap)

    # f_max = 50
    # plt.ylim((0, f_max))
    plt.colorbar()
    # plt.clim(0, 20)
    # cmap.set_bad('grey')
    ax.set_xticks(sweep_axis + d_sweep/2.0) # otherwise they appear left of column
    ax.set_xticklabels(['{:.1f}'.format(v) for v in sweep_axis])
    ax.set_xlim((sweep_axis.min(), sweep_plotaxis.max()))
    ax.set_yticks(np.arange(0,50,1), minor=True)
    ax.set_yticks(np.arange(0,55,5), minor=False)
    ax.set_ylim((0, 50))
    # TODO: find out why one column doesn't show

    plt.ylabel('frequency (Hz)')
    plt.xlabel(sweep_var_legend)
    plt.suptitle('Evolution of PSD ({})'.format(sig_label))

In [ ]:
compare_psd_heatmap('STN_Vm', False)
compare_psd_heatmap('GPE_Vm', False)

## Entrainment Power

PSD peak magnitude at frequency of applied cortical bursting inputs.

In [ ]:
%%skip $without_ctx_bursting

def compare_entrainment(sig_label, input_freqs):
    """ Plot PSD magnitude at frequency of applied cuortical bursting """
    fig, ax = plt.subplots(figsize=(0.75*page_width, ax_height))
    sweep_vals = np.array(sorted(analysis_results.keys()))
    sweep_power = []
    for i, sweep_value in enumerate(sweep_vals):
        results = analysis_results[sweep_value]
        freqs, psd = results['PSD'][sig_label]
        freq_axis = freqs.magnitude
        f_input = input_freqs[i]
        i_entrain, = np.where(freq_axis == f_input)
        p_entrain = psd[i_entrain[0]]
        sweep_power.append(p_entrain)
    
    ax.plot(sweep_vals, sweep_power, '-')
    ax.plot(sweep_vals, sweep_power, 'r+')
    vmin, vmax = sweep_vals.min(), sweep_vals.max()
    ax.set_xticks(np.arange(vmin,vmax,.2), minor=False)
    # ax.set_xticks(np.arange(vmax,vmax,1), minor=True)
    # ax.set_xlim((0, 50))
    ax.set_ylabel('Power ({})'.format(psd.units))
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True)
    ax.set_title('PSD @ f_input ({})'.format(sig_label))

In [ ]:
%%skip $without_ctx_bursting

# f_inputs = [5.0 for i in range(len(analysis_results))]
f_inputs = [7, 8, 9, 12, 12, 12]
compare_entrainment('STN_Vm', input_freqs=f_inputs)
compare_entrainment('GPE_Vm', input_freqs=f_inputs)

## Peak Power

Magnitude of the largest PSD peak, assumed to be the frequency of bursting in the STN - GPe loop.

In [ ]:
def plot_max_psd(sig_label):
    """ Plot magnitude of largest PSD peak """
    fig, ax = plt.subplots(figsize=(0.75*page_width, ax_height))
    sweep_vals = np.array(sorted(analysis_results.keys()))
    sweep_power = []
    for i, sweep_value in enumerate(sweep_vals):
        results = analysis_results[sweep_value]
        freqs, psd = results['PSD'][sig_label]
        sweep_power.append(psd.magnitude.max())
    
    ax.plot(sweep_vals, sweep_power, '-')
    ax.plot(sweep_vals, sweep_power, 'r+')
    vmin, vmax = sweep_vals.min(), sweep_vals.max()
    ax.set_xticks(np.arange(vmin,vmax,.2), minor=False)
    # ax.set_xticks(np.arange(vmax,vmax,1), minor=True)
    # ax.set_xlim((0, 50))
    ax.set_ylim((0, 1.1*max(sweep_power)))
    ax.set_ylabel('Power ({})'.format(psd.units))
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True)
    ax.set_title('PSD max peak ({})'.format(sig_label))

def compare_max_psd(sig_labels):
    """ Plot magnitude of largest PSD peak """
    fig, ax = plt.subplots(figsize=(0.75*page_width, ax_height))
    
    sweep_vals = np.array(sorted(analysis_results.keys()))
    all_sweep_power = []
    
    colors = 'bgcm'
    for i, sig_label in enumerate(sig_labels):
        sweep_power = []
        for sweep_value in sweep_vals:
            results = analysis_results[sweep_value]
            freqs, psd = results['PSD'][sig_label]
            sweep_power.append(psd.magnitude.max())
        all_sweep_power.extend(sweep_power)

        ax.plot(sweep_vals, sweep_power, '-', color=colors[i], label="$PSD_{max}$" + " ({sig})".format(sig=sig_label))
        ax.plot(sweep_vals, sweep_power, 'r+')

    vmin, vmax = sweep_vals.min(), sweep_vals.max()
    x_dtick = 10.0 # SETPARAM: spacing for x-axis ticks
    ax.set_xticks(np.arange(vmin, vmax, x_dtick), minor=False)
    # ax.set_xticks(np.arange(vmax,vmax,1), minor=True)
    ax.set_xlim((0, 100))
    ax.set_ylim((0, 1.1*max(all_sweep_power)))
    ax.set_ylabel('Power ({})'.format(psd.units))
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True)
    ax.set_title('PSD max peak')
    ax.legend()
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel

In [ ]:
compare_max_psd(('STN_Vm', 'GPE_Vm'))
# plot_max_psd('STN_Vm')
# plot_max_psd('GPE_Vm')

## Subband Power

Summed PSD power in different frequency bands.

In [ ]:
def compare_subband_power(sig_label):
    """ Plot summed power in different subbands """
    fig, ax = plt.subplots(figsize=(0.75*page_width, ax_height))
    sweep_vals = np.array(sorted(analysis_results.keys()))
    sweep_Psum = {} # map band (x,y) -> Psum
    # collect data
    for i, sweep_value in enumerate(sweep_vals):
        results = analysis_results[sweep_value]
        bands, Psum = results['PSD_subband_power'][sig_label]
        for j, band in enumerate(bands):
            sweep_Psum.setdefault(band, []).append(Psum[j])
    # plot collected data
    import matplotlib
    bands = sweep_Psum.keys()
    lower, upper = zip(*bands)
    cNorm  = matplotlib.colors.Normalize(vmin=min(lower), vmax=max(lower))
    cmapping = matplotlib.cm.ScalarMappable(norm=cNorm, cmap=plt.get_cmap('cool'))
    bands_sorted = sorted(bands, key=lambda b: b[0])
    for band in bands_sorted:
        Psums = sweep_Psum[band]
        ax.plot(sweep_vals, Psums, '-', color=cmapping.to_rgba(band[0]), label='{} Hz'.format(band))
    vmin, vmax = sweep_vals.min(), sweep_vals.max()
    d1, d2 = .2, .1 # tick increments
    ax.set_xticks(np.arange(vmin,d1*(vmax//d1+1),d1), minor=False)
    # ax.set_xticks(np.arange(vmin,1*(vmax//1+1),1), minor=True)
    ax.set_xlim((vmin, vmax))
    ax.set_ylabel('Power (mV^2/Hz)')
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True)
    ax.legend()
    ax.set_title('sum(PSD) in sub-bands ({})'.format(sig_label))
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel

In [ ]:
compare_subband_power('STN_Vm')
compare_subband_power('GPE_Vm')

# Spectrogram

## Peak Power - Instantaneous

Note that this might give a different picture than comparing simulations with PSD alone: a simulation with periodic synchronization may have higher transient peaks, whereas the PSD (average over time) is lower.

In [ ]:
# (freqs[0:int(50/df)], t, Sxx[:,0:int(50/df)])
# Syy = Sxx[:, int(5000.0/t_res):]

# set temporal resolution of PSD from value in synchrony_analysis_auto.ipynb
# t_res = 20.0 # ms

def compare_max_spectrogram(sig_label, band=(4.0, 30.0), t_start=5000.0):
    """ Plot magnitude of largest spectrogram peak """
    fig, ax = plt.subplots(figsize=(0.75*page_width, ax_height))
    sweep_vals = np.array(sorted(analysis_results.keys()))
    sweep_power = []
    for i, sweep_value in enumerate(sweep_vals):
        results = analysis_results[sweep_value]
        freqs, t, Sxx = results['spectrogram'][sig_label]
        # only look at power in sub-band
        df = freqs[1] - freqs[0]
        f_slice = np.s_[int(band[0]/df):(int(band[1]/df)+1)]
        # first 5000 ms already cut when saving
        # dt = t[1] - t[0]
        # t_slice = np.s_[int(t_start/dt):]
        t_slice = np.s_[:]
        S_max = Sxx[f_slice, t_slice].max()
        sweep_power.append(S_max)
    
    ax.plot(sweep_vals, sweep_power, '-')
    ax.plot(sweep_vals, sweep_power, 'r+')
    # ax.set_xticks(np.arange(0,5*(sweep_vals.max()//5+1),5), minor=False)
    # ax.set_xticks(np.arange(0,1*(sweep_vals.max()//1+1),1), minor=True)
    # ax.set_xlim((0, 50))
    ax.set_ylim((0, 1.1*max(sweep_power)))
    ax.set_ylabel('Power (mV^2/Hz)')
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True)
    ax.set_title('Spectrogram max peak ({})'.format(sig_label))
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel

In [ ]:
compare_max_spectrogram('STN_Vm')
compare_max_spectrogram('GPE_Vm')

# Synchronization

## Mean & Variability

In [ ]:
def compare_synchronization(sig_label):
    """ Compare distribution of Morgera-index values (mean, median, std) """
    sweep_vals = np.array(sorted(analysis_results.keys()))
    mean_M, std_M = [], []
    M_datasets = []
    for i, sweep_value in enumerate(sweep_vals):
        results = analysis_results[sweep_value]
        t, M = results['Morgera_index'][sig_label]
        mean_M.append(np.mean(M))
        std_M.append(np.std(M))
        M_datasets.append(M)

    mean_M = np.array(mean_M)
    std_M = np.array(std_M)
    
    # Plot continuous line with confidence intervals
    fig, ax = plt.subplots(figsize=(0.75*page_width, ax_height))
    plt.plot(sweep_vals, mean_M)
    plt.plot(sweep_vals, mean_M, 'r+')
    plt.plot(sweep_vals, mean_M+std_M, 'g_')
    plt.plot(sweep_vals, mean_M-std_M, 'g_')
    plt.fill_between(sweep_vals, mean_M+std_M, mean_M-std_M, color='k', alpha=.5)

    vmin, vmax = sweep_vals.min(), sweep_vals.max()
    ax.set_xticks(sweep_vals, minor=False)
    ax.set_xticks(np.arange(vmin,1*(vmax//1+1),1), minor=True)
    ax.set_xlim((vmin, vmax))
    ax.set_yticks(np.arange(0,1.1,0.1), minor=False)
    ax.set_ylim((0, 1))
    ax.set_ylabel('M (0-1)')
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True, which='major', axis='y')
    ax.set_title('Morgera index ({})'.format(sig_label))
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel
    
    # Plot boxplots
    fig, ax = plt.subplots(figsize=(0.75*page_width, ax_height))
    bp = ax.boxplot(M_datasets, 0, 'g+')
    ax.set_xticklabels(sweep_vals)
    ax.set_yticks(np.arange(0,1.1,0.1), minor=False)
    ax.set_ylim((0, 1))
    ax.set_ylabel('M (0-1)')
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True, which='major', axis='y')
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel

In [ ]:
compare_synchronization('STN_Vm')

In [ ]:
compare_synchronization('GPE_Vm')

## Periodicity

Can use morgera index, or summed spectrogram sub-band power over time.

In [ ]:
%%javascript
// require(["base/js/namespace"],function(Jupyter) {
//     Jupyter.notebook.save_checkpoint();
// });

In [ ]:
# thisfile = 'sweep_analysis.ipynb'
# parent_dir, _ = os.path.split(output_dirs[-1])
# !jupyter nbconvert $thisfile --template=toc2 --output-dir=$outputs